pip install openai faiss-cpu sentence-transformers

In [10]:
import os
import faiss
from openai import OpenAI
from sentence_transformers import SentenceTransformer
import numpy as np

# 初始化
embed_model = SentenceTransformer("shibing624/text2vec-base-chinese")
client = OpenAI(api_key="xxx")

# 讀取 FAQ 文字檔（每行一筆）
def load_faq_from_txt(file_path):
    with open(file_path, 'r', encoding='utf-8') as f:
        return [line.strip() for line in f if line.strip()]

# 載入 FAQ 文件
faq_docs = load_faq_from_txt("faq.txt")

# 建立向量索引
faq_embeddings = embed_model.encode(faq_docs, convert_to_numpy=True)
index = faiss.IndexFlatL2(faq_embeddings.shape[1])
index.add(faq_embeddings)

# 檢索
def retrieve_faq(query, top_k=1):
    query_emb = embed_model.encode([query])
    _, indices = index.search(query_emb, top_k)
    return [faq_docs[i] for i in indices[0]]

# LLM 回答生成
def generate_answer(query, context):
    prompt = f"請根據以下 FAQ 內容回答問題：\n\n{context}\n\n問題：{query}\n\n回答："
    response = client.chat.completions.create(
        model="gpt-3.5-turbo",
        messages=[{"role": "user", "content": prompt}]
    )
    return response.choices[0].message.content.strip()

# 主流程
if __name__ == "__main__":
    print("歡迎使用 AI 智慧客服，輸入問題來試試看（輸入 q 離開）")
    while True:
        question = input("\n請輸入問題：")
        if question.lower() == 'q':
            break
        context = "\n".join(retrieve_faq(question, top_k=2))
        print("🔍 檢索到的資料：",context)
        answer = generate_answer(question, context)
        print("\nAI 回答：", answer)


歡迎使用 AI 智慧客服，輸入問題來試試看（輸入 q 離開）



請輸入問題： 狗可以吃嗎


🔍 檢索到的資料： Q: 狗狗可以吃嗎？ A: 可以的，我們的商品使用天然食材製成，無調味、無添加、無防腐劑，狗狗也適合食用。
Q: 人可以吃嗎？ A: 雖然原料為人食等級，但因為無調味，還是建議給寵物吃喔～

AI 回答： 可以的，狗狗可以吃這個商品，因為它是使用天然食材製成，無添加劑，無防腐劑，適合狗狗食用。



請輸入問題： 如何退貨


🔍 檢索到的資料： Q: 我收到的商品有瑕疵／有少寄，或想退換貨，怎麼辦？ A: 若收到商品有漏寄、瑕疵等問題，或需退換貨，請聯繫我們的 Line 或 Facebook 粉絲團，我們將協助處理。
Q: 我下錯單了／想取消訂單，怎麼辦？ A: 若想取消或修改訂單，請盡快聯繫我們，可留言於訂單頁或聯絡官方 Line，我們將協助處理。

AI 回答： 若需要退換貨，請聯繫我們的 Line 或 Facebook 粉絲團，我們將協助處理。


...


請輸入問題： 香港可以買嗎


🔍 檢索到的資料： Q: 我在國外可以買嗎？ A: 我們目前僅提供香港配送，運費依重量計算。其他國家因肉類產品入境限制，暫不開放寄送。
Q: 請問有實體店面嗎？ A: 目前好味小姐以官網銷售為主。合作店家資訊可以參考「合作店家的頁面資訊」，各店商品銷售狀況不一，前往購買前可以先與店家聯繫是否有現貨喔～

AI 回答： 可以，我們目前僅提供香港配送。



請輸入問題： 日本可以送嗎


🔍 檢索到的資料： Q: 生日購物金怎麼發送？ A: 將於生日當月 1 號發送至會員帳號，請確認註冊時已填寫生日，生日購物金需於發放日起 30 天內使用完畢，期限依會員頁面顯示為準。
Q: 我在國外可以買嗎？ A: 我們目前僅提供香港配送，運費依重量計算。其他國家因肉類產品入境限制，暫不開放寄送。

AI 回答： 暫時不行，因為我們目前僅提供香港配送，其他國家因肉類產品入境限制，暫不開放送貨服務。



請輸入問題： q
